In [1]:
!pip install langgraph>=0.5.4

In [8]:
!pip install langchain==0.3

  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached sqlalchemy-2.0.41-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.12.14-cp310-cp310-win_amd64.whl.metadata (7.9 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.3-cp310-cp310-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp310-cp310-win_amd64.whl.metadata (12 kB)
  U

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 0.5.2 requires langchain-core>=0.3.67, but you have langchain-core 0.3.63 which is incompatible.


In [11]:
pip install -U langchain-google-vertexai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.0 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.8 which is incompatible.



  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached langchain_core-0.3.70-py3-none-any.whl.metadata (5.8 kB)
  Using cached numexpr-2.11.0-cp310-cp310-win_amd64.whl.metadata (9.2 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached google_cloud_bigquery-3.35.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached google_cloud_resource_manager-1.14.2-py3-none-any.whl.metadata (9.6 kB)
  Using cached shapely-2.1.1-cp310-cp310-win_amd64.whl.metadata (7.0 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached grpcio-1.73.1-cp310-cp310-win_amd64.whl.metadata (4.0 kB)
  Using cached grpci

In [2]:
from langgraph.prebuilt import ToolNode, tools_condition

In [1]:
!pip show langgraph
# You should see Version: 0.5.4 (or higher)


Name: langgraph
Version: 0.5.4
Summary: Building stateful, multi-actor applications with LLMs
Home-page: 
Author: 
Author-email: 
License-Expression: MIT
Location: c:\users\prana\anaconda3\envs\test\lib\site-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk, pydantic, xxhash
Required-by: 


In [14]:
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START
from typing_extensions import TypedDict

# Step 1: Define your shared state type
class State(TypedDict):
    messages: list

# Step 2: Define your tool using @tool
@tool
def get_stock_price(symbol: str) -> str:
    """Returns mock stock price for a given symbol."""
    prices = {"AAPL": 150.0, "GOOG": 2800.0}
    return str(prices.get(symbol.upper(), 0.0))

# Step 3: Bind the tool to the LLM
llm = init_chat_model("gemini-2.0-flash")
llm_with_tools = llm.bind_tools([get_stock_price])

# Step 4: Node to call LLM
def call_llm(state: State) -> State:
    message = llm_with_tools.invoke(state["messages"])
    return {"messages": state["messages"] + [message]}

# Step 5: Define ToolNode
tools_node = ToolNode([get_stock_price])

# Step 6: Build the graph
graph = StateGraph(State)
graph = StateGraph(MyStateSchema)

graph.add_node("chat", chat_node)
graph.add_node("tools", tools_node)  # ✅ Required
graph.set_entry_point("chat")
graph.set_finish_point("chat")

def tools_condition(state):
    if should_use_tool(state):  # some logic
        return "tools"  # must match exactly
    return "chat"

graph.add_conditional_edges("chat", tools_condition)
graph.add_edge("tools", "chat")

agent = graph.compile()
response = agent.invoke({"messages": [HumanMessage(content="What's the AAPL stock price?")]})
print(response["messages"][-1].content)



ValueError: At 'chat' node, 'tools_condition' branch found unknown target 'tools'